## Utilisation dans Azure d'un dataset

In [3]:
import pandas as pd
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

# le data set est d'abord créé dans le menu Asset / Data / Create
# Actifs multimedias / Données / Créer (en français)

ml_client = MLClient.from_config(credential=DefaultAzureCredential())
data_asset = ml_client.data.get("titanic_data", version="1")

df_original = pd.read_csv(data_asset.path)
df_original.head()

Found the config file in: /config.json


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Nouvelle version du dataset

In [6]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

df_cleaned = df_original.drop(columns=["Cabin"], errors="ignore")

# Sauvegarde locale du nouveau CSV
cleaned_path = "titanic_data_cleaned.csv"
df_cleaned.to_csv(cleaned_path, index=False)

new_version = ml_client.data.get("titanic_data", version="2")
if new_version :
    print(f"titanic_data version 2 allready exists")

else : 
# Création de la nouvelle version du Data Asset (v2)
    new_data_asset = Data(
        path=cleaned_path,
        type=AssetTypes.URI_FILE,
        name="titanic_data",  # même nom
        version="2",              # nouvelle version
        description="Version nettoyée sans la colonne Cabin"
    )

    ml_client.data.create_or_update(new_data_asset)
    print(f"titanic_data version 2 created")


titanic_data version 2 allready exists


## Entraîner un modèle et consigner les résultats obtenus avec MLflow dans Azure ML

In [7]:
import pandas as pd
import mlflow
import mlflow.sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

# Connexion + chargement du dataset v2
ml_client = MLClient.from_config(credential=DefaultAzureCredential())
data_asset = ml_client.data.get(name="titanic_data_set", version="2")
df = pd.read_csv(data_asset.path)

# Préparation des données (exemple simple)
df = df.dropna()
X = df.drop(columns=["Survived"])
y = df["Survived"]

# Dummy encoding si besoin (ex: pour les colonnes catégorielles)
X = pd.get_dummies(X)

# Split train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Démarrer un run MLflow
with mlflow.start_run():
    model = LogisticRegression(max_iter=200)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1]

    # Métriques
    f1 = f1_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_proba)

    # Log avec MLflow
    mlflow.log_param("model_type", "LogisticRegression")
    mlflow.log_metric("f1_score", f1)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("roc_auc", roc_auc)

    mlflow.sklearn.log_model(model, "model")

    print(f"✅ Modèle loggé avec MLflow - F1: {f1:.3f}, Precision: {precision:.3f}, Recall: {recall:.3f}, AUC: {roc_auc:.3f}")


Found the config file in: /config.json
/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/azureml/mlflow/_protos/aml_service_pb2.py:10: UserWarning: google.protobuf.service module is deprecated. RPC implementations should provide code generator plugins which generate code specific to the RPC implementation. service.py will be removed in Jan 2025
  from google.protobuf import service as _service
/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
2025/04/02 13:32:26 WARNING mlflow.models.model: Model

✅ Modèle loggé avec MLflow - F1: 0.745, Precision: 0.872, Recall: 0.651, AUC: 0.845
